In [1]:
import pandas as pd
import numpy as np

# For visualizations
import matplotlib.pyplot as plt

import scipy
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas_profiling
from pandas_profiling import ProfileReport

import re
# For handling string
import string
# For performing mathematical operations
import math




ModuleNotFoundError: No module named 'pandas_profiling'

In [ ]:
# Importing dataset
df=pd.read_csv('Datafiniti_Amazon.csv') 
print("Shape of data=>",df.shape)

#Reference:https://www.analyticsvidhya.com/blog/2020/04/beginners-guide-exploratory-data-analysis-text-data/

In [ ]:
df.info()

In [ ]:
list(df)
df.shape
df.info()
df.describe().transpose()
df.head(n=20)
df.tail()
pd.isna(df)
df.corr()

In [ ]:
#Selecting columns
df=df[['name','reviews.text','reviews.doRecommend','reviews.numHelpful']]


In [ ]:
#Check for null values 
df.dropna(inplace=True)
df.isnull().sum()


In [ ]:
#Word Count of each review
df['word_count'] = df['reviews.text'].apply(lambda x: len(str(x).split(" ")))
df[['reviews.text','word_count']].head()

In [ ]:
#Average word count of each review

sum(df['word_count'])/len(df['reviews.text'])

In [ ]:
#Character Count of each review
df['char_count'] = df['reviews.text'].str.len() ## this also includes spaces
df[['reviews.text','char_count']].head()
plt.hist(df['char_count'])
plt.show()

In [ ]:
#Average Length of words in Review 0 
sum(len(df['reviews.text'][0]) for word in df['reviews.text'][0])/len(df['reviews.text'][0])

In [ ]:
#Average word size of each review

def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

df['avg_word'] = df['reviews.text'].apply(lambda x: avg_word(x))
df[['reviews.text','avg_word']].head()

In [ ]:
df['name'].unique()
print('Number of products=>',len(df['name'].unique()))

In [ ]:
# Keep products which have a count more than 500 
df=df.groupby('name').filter(lambda x:len(x)>500).reset_index(drop=True)
print('Number of products=>',len(df['name'].unique()))

In [ ]:
#Convert columns to int so they can be used to build a model later 
df['reviews.doRecommend']=df['reviews.doRecommend'].astype(int)
df['reviews.numHelpful']=df['reviews.numHelpful'].astype(int)

In [ ]:
#Cleaning Data


In [ ]:
df['name'].unique()


In [ ]:
# Remove ,,, from entries in name column
df['name'] = df['name'].str.replace(r'\,,,','')

df['name'].unique()

In [ ]:
#Check reviews. text column
print(df['reviews.text'][0])

In [ ]:
#Check first 30 reviews 
i=0
while i<50:
    print ("Review Number ",i)
    print()
    print(df['reviews.text'][i])
    print()
    i=i+1
    


In [ ]:
#Remove contractions from text reviews
# Dictionary of English Contractions
contractions_dict = { "It's":"It is","it's":"it is", "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

# Expanding Contractions in the reviews
df['reviews.text']=df['reviews.text'].apply(lambda x:expand_contractions(x))

In [ ]:
#Check first 30 reviews 
i=0
while i<50:
    print ("Review Number ",i)
    print()
    print(df['reviews.text'][i])
    print()
    i=i+1
    

In [ ]:
print(df['reviews.text'][0])

In [ ]:
df['reviews.text']=df['reviews.text'].str.lower()


In [ ]:
df

In [ ]:
#Alternative to lowercase letters
df['cleaned']=df['reviews.text'].apply(lambda x: x.lower())

In [ ]:
#Remove digits and words containing digits 
df['reviews.text']=df['reviews.text'].apply(lambda x: re.sub('\w*\d\w*','', x))


In [ ]:
#Remove Punctuations
df['reviews.text']=df['reviews.text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))


In [ ]:
#Check first 30 reviews after cleaning 
i=0
while i<50:
    print ("Review Number ",i)
    print()
    print(df['reviews.text'][i])
    print()
    i=i+1

In [ ]:
import nltk
nltk.download()

In [ ]:
Z=df['reviews.text']
print(Z)

In [ ]:
#check count of StopWords
from nltk.corpus import stopwords
stop = stopwords.words('english')

df['stopwords'] = df['reviews.text'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['reviews.text','stopwords']].head()

plt.hist(df['stopwords'])

plt.show()

In [ ]:
#Commonly occuring words
freq = pd.Series(' '.join(df['reviews.text']).split()).value_counts()[:10]
freq

In [ ]:
#removal of commonly occuring irrelevant words
freq = list(freq)
df['reviews.text'] = df['reviews.text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df['reviews.text'].head()

In [ ]:
#Count of rare words

freq_rare = pd.Series(' '.join(df['reviews.text']).split()).value_counts()[-100:]
freq_rare

In [ ]:
#removal of rarely occuring irrelevant words
freq_rare = list(freq_rare)
df['reviews.text'] = df['reviews.text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df['reviews.text'].head()

In [ ]:
#Spell Correction(Time consuming), Try on first 5 reviews
from textblob import TextBlob
df['reviews.text_Corrected Spelling']=df['reviews.text'][1:5].apply(lambda x: str(TextBlob(x).correct()))
df.head(5)

In [ ]:
# Import stopwords with nltk.
from nltk.corpus import stopwords
stop = stopwords.words('english')

# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
df['reviews.text.nostopwords'] = df['reviews.text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(df)


In [ ]:
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

df['reviews.text.nostopwords.lemmatized2'] = df['reviews.text.nostopwords'].apply(lambda x:lemmatize_text(x))


In [ ]:
df

In [ ]:
pip install textblob


In [ ]:
#Counting number of words using TextBlob
from textblob import TextBlob
TextBlob(df['reviews.text.nostopwords'][1]).word_counts


In [ ]:
#Lemmatization

from textblob import Word
df['reviews.text.nostopwords.LEMMATIZED'] = df['reviews.text.nostopwords'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df['reviews.text.nostopwords.LEMMATIZED'].head()

In [ ]:
#n-gram using TextBlob

TextBlob(df['reviews.text.nostopwords.LEMMATIZED'][0]).ngrams(3)

In [ ]:
#Scikit learn implementation(Tfidf vectorizer)

from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
#TFID BOW Vectorizer

vec_idf=TfidfVectorizer()

In [ ]:
#Fitting the vectorizer on the data to create vocabulary

w=vec_idf.fit(df['reviews.text.nostopwords.LEMMATIZED'])

In [ ]:
#Vocabulary 

vec_idf.vocabulary_

In [ ]:
#TFIDF Score calculation
#Output would be sparse matrix with TFIDF score calculation 

z=w.transform(df['reviews.text.nostopwords.LEMMATIZED'])

In [ ]:
#Converting sparse matrix into array

print(z.toarray())

# Document Term Matrix (Another Approach)

In [ ]:
# Group stopwords and lemmatized reviews according to products
df_grouped=df[['name','reviews.text.nostopwords.LEMMATIZED']].groupby(by='name').agg(lambda x:' '.join(x))
df_grouped

In [ ]:
# Creating Document Term Matrix
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(analyzer='word')
data=cv.fit_transform(df_grouped['reviews.text.nostopwords.LEMMATIZED'])
df_dtm = pd.DataFrame(data.toarray(), columns=cv.get_feature_names())
df_dtm.index=df_grouped.index
df_dtm.head(3)

In [ ]:
df_dtm.head(3)

# Exploratory Analysis

In [ ]:
pip install wordcloud

In [ ]:
# Importing wordcloud for plotting word clouds and textwrap for wrapping longer text
from wordcloud import WordCloud
from textwrap import wrap

# Function for generating word clouds
def generate_wordcloud(data,title):
  wc = WordCloud(width=400, height=330, max_words=150,colormap="Dark2").generate_from_frequencies(data)
  plt.figure(figsize=(10,8))
  plt.imshow(wc, interpolation='bilinear')
  plt.axis("off")
  plt.title('\n'.join(wrap(title,60)),fontsize=13)
  plt.show()
  
# Transposing document term matrix
df_dtm=df_dtm.transpose()

# Plotting word cloud for each product
for index,product in enumerate(df_dtm.columns):
  generate_wordcloud(df_dtm[product].sort_values(ascending=False),product)

# Sentiment Analysis

In [ ]:
from textblob import TextBlob
df['polarity']=df['reviews.text.nostopwords.LEMMATIZED'].apply(lambda x:TextBlob(x).sentiment.polarity)

In [ ]:
#Reviews with Highest Polarity 
print("3 Random Reviews with Highest Polarity:")
for index,review in enumerate(df.iloc[df['polarity'].sort_values(ascending=False)[:3].index]['reviews.text']):
  print('Review {}:\n'.format(index+1),review)

In [ ]:
#Reviews with Lowest Polarity 
print("3 Random Reviews with Lowest Polarity:")
for index,review in enumerate(df.iloc[df['polarity'].sort_values(ascending=True)[:3].index]['reviews.text']):
  print('Review {}:\n'.format(index+1),review)

In [ ]:
#Plotting Polarity of different products
product_polarity_sorted=pd.DataFrame(df.groupby('name')['polarity'].mean().sort_values(ascending=True))

plt.figure(figsize=(16,8))
plt.xlabel('Polarity')
plt.ylabel('Products')
plt.title('Polarity of Different Amazon Product Reviews')
polarity_graph=plt.barh(np.arange(len(product_polarity_sorted.index)),product_polarity_sorted['polarity'],color='purple',)

# Writing product names on bar
for bar,product in zip(polarity_graph,product_polarity_sorted.index):
  plt.text(0.005,bar.get_y()+bar.get_width(),'{}'.format(product),va='center',fontsize=11,color='white')

# Writing polarity values on graph
for bar,polarity in zip(polarity_graph,product_polarity_sorted['polarity']):
  plt.text(bar.get_width()+0.001,bar.get_y()+bar.get_width(),'%.3f'%polarity,va='center',fontsize=11,color='black')
  
plt.yticks([])
plt.show()

In [ ]:
#Plotting recommendation of different products
recommend_percentage=pd.DataFrame(((df.groupby('name')['reviews.doRecommend'].sum()*100)/df.groupby('name')['reviews.doRecommend'].count()).sort_values(ascending=True))

plt.figure(figsize=(16,8))
plt.xlabel('Recommend Percentage')
plt.ylabel('Products')
plt.title('Percentage of reviewers recommended a product')
recommend_graph=plt.barh(np.arange(len(recommend_percentage.index)),recommend_percentage['reviews.doRecommend'],color='green')

# Writing product names on bar
for bar,product in zip(recommend_graph,recommend_percentage.index):
  plt.text(0.5,bar.get_y()+0.4,'{}'.format(product),va='center',fontsize=11,color='white')

# Writing recommendation percentage on graph
for bar,percentage in zip(recommend_graph,recommend_percentage['reviews.doRecommend']):
  plt.text(bar.get_width()+0.5,bar.get_y()+0.4,'%.2f'%percentage,va='center',fontsize=11,color='black')

plt.yticks([])
plt.show()

In [ ]:
#Checking readability of reviews


In [ ]:
pip install textstat

In [ ]:
#Using textstat to determine readability, complexity and grade level of particular review
import textstat
df['dale_chall_score']=df['reviews.text'].apply(lambda x: textstat.dale_chall_readability_score(x))
df['flesh_reading_ease']=df['reviews.text'].apply(lambda x: textstat.flesch_reading_ease(x))
df['gunning_fog']=df['reviews.text'].apply(lambda x: textstat.gunning_fog(x))

print('Dale Chall Score of upvoted reviews=>',df[df['reviews.numHelpful']>1]['dale_chall_score'].mean())
print('Dale Chall Score of not upvoted reviews=>',df[df['reviews.numHelpful']<=1]['dale_chall_score'].mean())

print('Flesch Reading Score of upvoted reviews=>',df[df['reviews.numHelpful']>1]['flesh_reading_ease'].mean())
print('Flesch Reading Score of not upvoted reviews=>',df[df['reviews.numHelpful']<=1]['flesh_reading_ease'].mean())

print('Gunning Fog Index of upvoted reviews=>',df[df['reviews.numHelpful']>1]['gunning_fog'].mean())
print('Gunning Fog Index of not upvoted reviews=>',df[df['reviews.numHelpful']<=1]['gunning_fog'].mean())

In [ ]:
#Checking if upvoted and not upvoted reviews are easility understandable by people who completed certain grade school
df['text_standard']=df['reviews.text'].apply(lambda x: textstat.text_standard(x))

print('Text Standard of upvoted reviews=>',df[df['reviews.numHelpful']>1]['text_standard'].mode())
print('Text Standard of not upvoted reviews=>',df[df['reviews.numHelpful']<=1]['text_standard'].mode())


In [ ]:
#Checking reading time for a review (Average adults reads 250 words per minute)

df['reading_time']=df['reviews.text'].apply(lambda x: textstat.reading_time(x))

print('Reading Time of upvoted reviews=>',df[df['reviews.numHelpful']>1]['reading_time'].mean())
print('Reading Time of not upvoted reviews=>',df[df['reviews.numHelpful']<=1]['reading_time'].mean())

#People find longer reviews helpful

In [ ]:
#conclusion from EDA on Amazon Product Revies

#Customers love products from Amazon. They find them a great purchase and easy to use
#Amazon needs to work on the Fire Kids Edition Tablet because it has the most negative reviews. It is also the least recommended product
#The majority of reviews are written in simple English and are easily understandable by anyone who has 5th or 6th grade of school
#The reading time of helpful reviews is twice that of non-helpful reviews which means people find longer reviews helpful
 